# How to use the framework


In case that FrImCla is not installed in your system, the first task consist in installing using pip.

In [1]:
!pip install frimcla

To begin, we have to import all the classes that we will need to be able to use our framework.

In [2]:
import warnings
import time
import json
import argparse
from frimcla.utils.conf import Conf
from imutils import paths
from frimcla.index_features import generateFeatures
from frimcla.StatisticalComparison import statisticalComparison, majorityVoting
from frimcla.train import train
from frimcla.prediction import prediction,predictionEnsemble
warnings.simplefilter(action="ignore", category=FutureWarning)

Using TensorFlow backend.


### Configuring the dataset path

First of all we have to know the path which we have our dataset. The dataset must have a folder for each class that we want to predict. 

In [3]:
!wget "https://drive.google.com/uc?id=1ZApIHn-EvoQ6sdgJWhgfM3rmEEXWvXRJ&export=download&authuser=0" -O mias.zip
!unzip mias.zip

datasetPath = "./Mias"

--2019-03-19 16:27:11--  https://drive.google.com/uc?id=1ZApIHn-EvoQ6sdgJWhgfM3rmEEXWvXRJ&export=download&authuser=0
Resolving drive.google.com (drive.google.com)... 172.217.212.138, 172.217.212.139, 172.217.212.102, ...
Connecting to drive.google.com (drive.google.com)|172.217.212.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/uc?id=1ZApIHn-EvoQ6sdgJWhgfM3rmEEXWvXRJ&export=download [following]
--2019-03-19 16:27:11--  https://drive.google.com/uc?id=1ZApIHn-EvoQ6sdgJWhgfM3rmEEXWvXRJ&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jdmkem3ge3qi6phhk2gmkct5isrnfago/1553011200000/05147614325801676241/*/1ZApIHn-EvoQ6sdgJWhgfM3rmEEXWvXRJ?e=download [following]
--2019-03-19 16:27:15--  https://doc-0g-3s-docs.googleusercontent.com/docs/secu

### Feature Extractor

In this step we decide the feature extractor models that we are going to use with our dataset. These models will extract the most important points of the images. Then we save the points and with the classifier models that we will choose after this, we will classify the images with the classes of the dataset. Each feature extractor model has a different way to collect the most important points and for this reason we have to compare the models, because there is not a model that always fits better with the datasets.

In [0]:
featureExtractors = [["resnet", "False"],["vgg16", "False"]]

Now that we have the feature extractor models we can execute the algorithm that collect the features of the dataset for each model. The only thing that we have to do is indicate the paths of the dataset and the output and the models that we want to use for the study. The verbose parameter is to indicate whether we want to appear information about the execution on console.

In [5]:
generateFeatures("./output", 32, datasetPath, featureExtractors, False)

[INFO] loading ['resnet', 'False']...
Instructions for updating:
Colocations handled automatically by placer.
102858752/102853048 [==============================] - 2s 0us/step
[WARN] minimum init buffer not reached - 2019-03-19 16:27:48.860938
[INFO] creating datasets... - 2019-03-19 16:27:48.861521
[INFO] writing un-empty buffers... - 2019-03-19 16:27:48.862509
[INFO] writing `image_ids` buffer - 2019-03-19 16:27:48.863047
[INFO] writing `features` buffer - 2019-03-19 16:27:48.864031
[INFO] compacting datasets... - 2019-03-19 16:27:48.868315
[INFO] old size of `image_ids`: 296; new size: 296 - 2019-03-19 16:27:48.868475
[INFO] old size of `features`: 296; new size: 296 - 2019-03-19 16:27:48.868951
[INFO] loading ['vgg16', 'False']...
553467904/553467096 [==============================] - 26s 0us/step
[WARN] minimum init buffer not reached - 2019-03-19 16:28:26.525212
[INFO] creating datasets... - 2019-03-19 16:28:26.525867
[INFO] writing un-empty buffers... - 2019-03-19 16:28:26.5268

This algorithm will create a set of files that contains the features of the images. Each file corresponds to a model of those indicated above. 

### Classification models

Once we have stored the features of the images, we have to choose the clasiffication models that we are going to use for the dataset. All these classifiers will be used for each feature extractor model to know which is the performance of every combination.

In [0]:
modelClassifiers = [ "MLP","SVM","KNN", "LogisticRegression", "GradientBoost", "RandomForest"]

With the classifiers chosen, now that we have to do is to carry out a statistical analysis. The analysis studies and compares every combination. Once the analysis has compared all the combinations gives us the best combination of feature extractor model and classifier model and all the combinations that have not significant differencies with the best result.  

### Performance measures and ensemble

We have to select a performance measure to know which is the performance of the algorithm. In this case, there are five different measures (accuracy, recall, precision, auroc and f1). The user have to select only one of the five measures. Accuracy is the default measure.

To improve the performance, the framework uses an ensemble technique called majority voting. This technique use all the models generated to predict the class of the image. The framework only saves the models that have a certain percentage of the measure chosen by the user. Then these models will be trained to classify the images. 

In [0]:
measure = "accuracy"

In [0]:
majorityVoting("./output", datasetPath, featureExtractors, modelClassifiers, measure, False)

### Training the models

Finally, we have to train the models. In this step, all the models that have been selected will be trained. But only models with more than 56% of the measure chosen previously. Thus, the framework prevents models with bad results from worsening the prediction. In this case we do not split the dataset in test and train data, we need all the images to train and improve the results of the model. 

In this function FrImCla asks the user if he/she wants a web application or not (Y if the user wants the webapp). This web application uses the best model selected by FrImCla for this problem. The application generated is very simple and only contains a text box and a button to predict the class of new images. FrImCla generates a zip file that must be decompressed to be used. Inside the folder the user has to open a new terminal in this path and run the following command:

`python3 FlaskApp/app.py`

The comand executes the web application in http://localhost:5000/ To use the application the user has to write open the link in his/her browser.


In [9]:
train("./output", datasetPath, 1)

[INFO] gathering train/test splits...
[INFO] tuning hyperparameters...
[INFO] best hyperparameters: {'activation': 'tanh', 'alpha': 0, 'learning_rate': 'adaptive', 'momentum': 0.9, 'solver': 'lbfgs'}
[INFO] dumping classifier...


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[INFO] best hyperparameters: {'kernel': 'linear', 'gamma': 0.0001, 'C': 1000}
[INFO] dumping classifier...


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[INFO] best hyperparameters: {'n_neighbors': 11}
[INFO] dumping classifier...
[INFO] best hyperparameters: {'C': 10000.0}
[INFO] dumping classifier...


...

[INFO] best hyperparameters: {'min_samples_leaf': 1, 'max_features': 10, 'max_depth': 3}
[INFO] dumping classifier...


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[INFO] best hyperparameters: {'min_samples_leaf': 1, 'max_features': 10, 'max_depth': None, 'criterion': 'entropy', 'bootstrap': False}
[INFO] dumping classifier...
[INFO] gathering train/test splits...
[INFO] tuning hyperparameters...


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr

[INFO] best hyperparameters: {'activation': 'logistic', 'alpha': 0, 'learning_rate': 'constant', 'momentum': 0.9, 'solver': 'lbfgs'}
[INFO] dumping classifier...
[INFO] best hyperparameters: {'kernel': 'linear', 'gamma': 0.001, 'C': 1000}
[INFO] dumping classifier...
[INFO] best hyperparameters: {'n_neighbors': 5}
[INFO] dumping classifier...
[INFO] best hyperparameters: {'C': 1000.0}
[INFO] dumping classifier...
[INFO] best hyperparameters: {'min_samples_leaf': 1, 'max_features': 3, 'max_depth': None}
[INFO] dumping classifier...
[INFO] best hyperparameters: {'min_samples_leaf': 1, 'max_features': 10, 'max_depth': 3, 'criterion': 'gini', 'bootstrap': False}
[INFO] dumping classifier...
Do you want to generate a web app to classify the images with the best combination? y/n
y


Once we have the model trained, we can predict the class of the new images.


If you want to download the web application you have to execute the following code.

In [0]:
from google.colab import files
files.download('./output/Mias/web.zip') 

### Prediction

Now, with the models trained we can predict the classes of our images. For this task, we have developed another algorithm to use the models. This execution will give us the predicted class of the image that we choose. 

In [15]:
datasetName = datasetPath[datasetPath.rfind("/")+1:]

with open("./output/" + datasetName +'/ConfModel.json') as data:
    datos = json.load(data)

extractors = datos["featureExtractors"]
classifiers = ["GradientBoost","RandomForest", "SVM","KNN","LogisticRegression", "MLP"]

imagePaths = "./Mias/NORMAL"
predictionEnsemble(featureExtractors, classifiers, imagePaths, "./output", datasetName)


[INFO] loading ['resnet', 'False']...
[INFO] loading ['vgg16', 'False']...
[INFO] class predicted for the image ./Mias/NORMAL/mdb234.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb237.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb112.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb210.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb047.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb133.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb162.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb129.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb255.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb297.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb064.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb192.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb029.jpg: 1
[INFO] class predicted for the image ./Mias/NORMAL/mdb022.jpg: 1
[INFO] class pr